In [ ]:
!pip install kafka-python

In [ ]:
import os
import getpass
import sys
import json
import base64
import kafka
import ssl
import time
import datetime
import matplotlib.pyplot as plt
import io
from PIL import Image

TEST_CAMERA_INPUT_EVENTSTREAMS_TOPIC = 'TestCameraInput'
TEST_CAMERA_ID = "TestCamera-0"
count = 0

In [ ]:
creds_string = getpass.getpass()
creds = json.loads(creds_string)


In [ ]:
# Connect to EventStreams, with our loaded credentials and attach to the requested Topic.
prod = None
while prod is None:
    try:
        prod = kafka.KafkaProducer(bootstrap_servers=creds["kafka_brokers_sasl"], \
                                   security_protocol="SASL_SSL", \
                                   sasl_mechanism="PLAIN", \
                                   sasl_plain_username=creds["user"], \
                                   sasl_plain_password=creds["api_key"], \
                                   ssl_cafile=ssl.get_default_verify_paths().cafile)

    except kafka.errors.NoBrokersAvailable:
        print("No Brokers Available. Retrying ...")
        time.sleep(1)
        prod = None


In [ ]:
with open(os.environ.get("DSX_PROJECT_DIR") + "/datasets/extra-testing/multi_color-7.png", "rb") as f:
    img = f.read()


In [ ]:
fi = io.BytesIO(img)
oi = Image.open(fi)

plt.close()
plt.imshow(oi)
plt.title("Original Image")
plt.show()

oi.close()


In [ ]:
val = {
    'camera': TEST_CAMERA_ID,
    'count': count,
    'timestamp': datetime.datetime.utcnow().isoformat() + 'Z',
    'image': base64.b64encode(img).decode('utf-8')
}
count += 1

prod.send(TEST_CAMERA_INPUT_EVENTSTREAMS_TOPIC, value=json.dumps(val).encode('utf-8'))
